In [2]:
import os
import pandas as pd
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID, STORED, KEYWORD

In [ ]:
csv_file_path = "../data/fullListDownloaded.csv"
dataframe = pd.read_csv(csv_file_path)

lists = ['keywords', 'keywordsmanual', 'category']
for l in lists:
    dataframe[l] = dataframe[l].fillna('')
    dataframe[l] = dataframe[l].apply(lambda x: x.split(';'))

dataframe = dataframe[['doi', 'article title', 'comment', 'keywordsmanual', 'filename',
       'keywords', 'category', 'authors', 'abstract', 'doi link',
       ]]
# replace nan with "null"
dataframe = dataframe.fillna("null")

In [4]:
file_path = "../txtFiles/final"
txtFinal = os.listdir(file_path)
txtFinal = [x.replace('.txt', '') for x in txtFinal]    

def read_text_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

dataframe['text'] = ''
for i in range(len(dataframe)):
    if dataframe['filename'][i] in txtFinal:
        file = os.path.join(file_path, dataframe['filename'][i] + '.txt')
        dataframe['text'][i] = read_text_file(file)
dataframe.head(2)

,doi,article title,comment,keywordsmanual,filename,keywords,category,authors,abstract,doi link,text
0,10.1038/s41467-019-11340-4,recent trend reversal for declining european s...,null,"[status of blue carbon, side-effects, seagrass]",delossantos_2019_RecentTrendReversal_10.1038_s...,"[eelgrass zostera marina, wasting disease, c...","[coastal, ecosystem, seagrass]","de los santos, cb; krause-jensen, d; alcoverro...","seagrass meadows, key ecosystems supporting fi...",https://doi.org/10.1038/s41467-019-11340-4,"\n: Seagrass meadows, key ecosystems supportin..."
1,10.1088/1748-9326/acdae9,carbon sequestration assessment and analysis i...,life-cycle analysis,"[carbon stored, carbon content, kelp, seaweed]",lian_2023_CarbonSequestrationAssessment_10.108...,"[greenhouse effect, blue carbon, carbon sequ...","[carbon cycle, ccs and carbon sequestration, ...","lian, ys; wang, r; zheng, jh; chen, wx; chang,...",methods for carbon sequestration are warranted...,https://doi.org/10.1088/1748-9326/acdae9,\n: Methods for carbon sequestration are warra...


In [47]:
schema = Schema(
    doi=ID(stored=True),
    title=TEXT(stored=True),
    comment=STORED(),
    keywordsmanual=KEYWORD(stored=True),
    filename=ID(stored=True),
    keywords=KEYWORD(stored=True),
    category=KEYWORD(stored=True),
    authors=TEXT(stored=True),
    abstract=TEXT(stored=True),
    doi_link=ID(stored=True),
    text=TEXT(stored=True),
)

# Create the index directory
if not os.path.exists("../indexWhoosh"):
    os.mkdir("../indexWhoosh")

# Create the index
ix = create_in("../indexWhoosh", schema)

# Open a writer for the index
writer = ix.writer()

# Add documents to the index
for i, row in dataframe.iterrows():
    writer.add_document(
        doi=row['doi'],
        title=row['article title'],
        comment=row['comment'],
        keywordsmanual=row['keywordsmanual'],
        filename=row['filename'],
        keywords=row['keywords'],
        category=row['category'],
        authors=row['authors'],
        abstract=row['abstract'],
        doi_link=row['doi link'],
        text=row['text'],
    )

writer.commit()


In [ ]:
# Search the index

from whoosh import index
from whoosh import index
from whoosh import qparser

def index_search(dirname, search_fields, search_query):
    ix = index.open_dir(dirname)
    schema = ix.schema
    
    og = qparser.OrGroup.factory(0.9)
    mp = qparser.MultifieldParser(search_fields, schema, group = og)

    
    q = mp.parse(search_query)
    
    
    with ix.searcher() as s:
        results = s.search(q, terms=True, limit = 10)
        print("Search Results: ")
        
        
        print(results[0:10])
    
results_dict = index_search("../indexWhoosh", ['title',], u"mangrove coastal exposure")